In [175]:
from xetrack import Reader
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# import plotly.io as pio
# pio.renderers.default='notebook'
WORKFLOW = 'append' # random, taxi
df = Reader('../output/stats.db').to_df()
# Filter last experiemnt
df = df[df['track_id']==df.tail(1)['track_id'].iloc[0]]
df = df[df['tech'] != 'shutil']
df = df[df['tech'] != 'm1']
df = df[df['step']>=0]
df['name'] = df['name'].str.replace('_',' ')
df['function'] = df['function'].str.replace('_',' ')
df['function'] = df['function'].str.replace(' new upload','').str.replace(' merged upload','')
df['mb/s'] = df['file_bytes']/df['time']
df.to_csv('../output/results.csv', index=False)
errors = set(df['error'].fillna('').values)
if len(errors) > 1:
    print(f"Errors: {errors}")
print(f"Steps: {df['step'].max()+1}")
print(f"Data size: {len(df)}")
px.bar(df, x='function', y='mb/s', color='tech').show()

Steps: 2
Data size: 3


In [176]:
px.bar(df, x='function', y='time', color='tech').show()

In [178]:
df[['name','bytes_sent']]

,name,bytes_sent
1416,gitxet new upload,201.390625
1417,pyxet new upload,12.370117
1418,gitxet new upload,26.583984


In [172]:
groups = df.groupby('function')
sums = groups.sum()
fig1 = px.pie(sums, values='time', names=sums.index)
fig1.update_traces(textposition='inside', textinfo='percent+label')

fig2 = go.Figure()
for name, group in groups:
    fig2.add_trace(go.Scatter(x=group['step'], y=group['time'], mode='lines', name=str(name)))
fig2.update_layout(xaxis_title='Step', yaxis_title='Time')

df['cumulative_time'] = df.groupby('function')['time'].cumsum()
fig3 = px.line(df, x='step', y='cumulative_time', color='function')
fig3.update_layout(title='Cumulative Lines of Time per Steps', xaxis_title='Steps', yaxis_title='Cumulative Time')

df['tech_cumulative_time'] = df.groupby('tech')['time'].cumsum()

fig4 = px.line(df, x='step', y='tech_cumulative_time', color='tech')
fig4.update_layout(title='Cumulative Lines of Time per Steps',
                  xaxis_title='Steps',
                  yaxis_title='Cumulative Time')

fig1.show()
fig3.show()
fig2.show()
fig4.show()

In [171]:
for step in set(df['step']):
    print(f"Step {step} took {df[df['step']==step]['time'].sum()/60:.2f} minutes")
print(f"total time is: {df['time'].sum()/(60*60):.2f} hours")

Step 0 took 1.67 minutes
Step 1 took 1.90 minutes
Step 2 took 0.38 minutes
total time is: 0.07 hours


In [86]:
times = df.groupby('function')['time'].mean()
ratios = times / times.min()
print(f"Average time:")
fig5 = px.pie(times, values='time', names=times.index)
fig5.update_traces(textposition='inside', textinfo='percent+label')
fig5.show()
ratios.sort_values()

Average time:


function
s3         1.000000
lakefs     1.198064
lfs git    1.363922
lfs s3     1.625733
pyxet      1.687559
dvc        2.010313
git-xet    2.070943
Name: time, dtype: float64